In [1]:
# Virginia gubernatorial elections

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import scipy
import scipy.optimize as opt
import datetime

In [31]:
import pymc as pm
import arviz as az

ImportError: cannot import name 'ResampleCompatible' from 'xarray.core.types' (C:\Users\adhva\anaconda3\Lib\site-packages\xarray\core\types.py)

In [35]:
!pip install --upgrade xarray

  Using cached xarray-2025.9.0-py3-none-any.whl.metadata (12 kB)
Using cached xarray-2025.9.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: xarray
    Found existing installation: xarray 2024.11.0
    Uninstalling xarray-2024.11.0:
      Successfully uninstalled xarray-2024.11.0


In [17]:
URL = 'https://docs.google.com/spreadsheets/d/1_y0_LJmSY6sNx8qd51T70n0oa_ugN50AVFKuJmXO1-s/export?format=csv&gid=101141872#gid=101141872'
polls = pd.read_csv(URL)
polls.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,cycle,office_type,seat_name,election_date,stage,party,answer,candidate_id,candidate_name,pct
0,82890,1802,VoteTXT,1990,Nevada Newsmakers,NaN,NaN,NaN,VoteTXT,Text-to-Web,...,2026,Governor,Governor,2026-11-03,general,REP,Lombardo,31014,Joe Lombardo,50.68
1,82890,1802,VoteTXT,1990,Nevada Newsmakers,NaN,NaN,NaN,VoteTXT,Text-to-Web,...,2026,Governor,Governor,2026-11-03,general,DEM,Ford,31013,Aaron D. Ford,29.70
2,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,REP,Sears,31081,Winsome Sears,26.00
3,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,DEM,Spanberger,31079,Abigail A. Spanberger,27.00
4,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,REP,Miyares,31080,Jason S. Miyares,22.00


In [19]:
polls.shape

(316, 39)

In [21]:
polls.columns.values

array(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'display_name', 'mode', 'state',
       'start_date', 'end_date', 'internal', 'partisan', 'question_id',
       'sample_size', 'population', 'subpopulation', 'population_full',
       'tracking', 'created_at', 'updated_at', 'notes', 'url',
       'url_article', 'url_topline', 'url_crosstab', 'race_id', 'cycle',
       'office_type', 'seat_name', 'election_date', 'stage', 'party',
       'answer', 'candidate_id', 'candidate_name', 'pct'], dtype=object)

In [23]:
polls_va = polls[polls['state'] == 'Virginia']
polls_va.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,cycle,office_type,seat_name,election_date,stage,party,answer,candidate_id,candidate_name,pct
2,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,REP,Sears,31081,Winsome Sears,26.0
3,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,DEM,Spanberger,31079,Abigail A. Spanberger,27.0
4,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,REP,Miyares,31080,Jason S. Miyares,22.0
5,84207,1497,co/efficient,2029,Founders Insight,NaN,NaN,NaN,co/efficient,IVR/Text,...,2025,Governor,Governor,2025-11-04,general,DEM,Spanberger,31079,Abigail A. Spanberger,26.0
6,88265,1773,Research America,682,University of Mary Washington Center for Leade...,NaN,NaN,NaN,Research America,Live Phone/Online Opt-In Panel,...,2025,Governor,Governor,2025-11-04,general,REP,Miyares,31080,Jason S. Miyares,39.0


In [25]:
polls_va.shape

(60, 39)

In [29]:
polls_va['mode'].value_counts()

mode
IVR/Text                                      15
Live Phone                                    15
Online Opt-In Panel                            7
Live Phone/Text-to-Web                         6
Live Phone/Online Opt-In Panel/Text-to-Web     6
Live Phone/Online Opt-In Panel                 4
Text-to-Web                                    3
IVR/Online Opt-In Panel/Text-to-Web            2
Name: count, dtype: int64